<a href="https://colab.research.google.com/github/svnty/blood-cell-classification/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test environment

Notebook designed using:

- OS Version: `macOS-14.7.6-arm64-arm-64bit-Mach-O`

- Python Version: `3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]`

- Current Working Directory: `/Users/admin/Projects/Programming/blood-cell-classification`

In [19]:
import os
import sys
import platform

print("OS Version:", platform.platform())
print("Python Version:", sys.version)
print("Current Working Directory:", os.getcwd())

OS Version: macOS-14.7.6-arm64-arm-64bit-Mach-O
Python Version: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]
Current Working Directory: /Users/admin/Projects/Programming/blood-cell-classification


# Install kaggle api

Install the Kaggle API library using pip.


In [ ]:
%pip install kaggle

Check if kaggle API key is available

In [20]:
from pathlib import Path

# The Kaggle API looks for credentials in this order:
# 1. Current working directory: ./kaggle.json
# 2. Home directory: ~/.kaggle/kaggle.json
# 3. Environment variables: KAGGLE_USERNAME and KAGGLE_KEY

# Check current directory first
current_dir_kaggle = Path.cwd() / 'kaggle.json'
home_dir_kaggle = Path.home() / '.kaggle' / 'kaggle.json'

if current_dir_kaggle.exists():
    print("Found kaggle.json in current directory")
    print(f"Location: {current_dir_kaggle}")

    # Set secure permissions (600 = read/write for owner only)
    current_dir_kaggle.chmod(0o600)
    print("Set secure permissions (600) on kaggle.json")
elif home_dir_kaggle.exists():
    print("Found kaggle.json in home directory")
    print(f"Location: {home_dir_kaggle}")

    # Set secure permissions (600 = read/write for owner only)
    home_dir_kaggle.chmod(0o600)
    print("Set secure permissions (600) on kaggle.json")
else:
    print("ERROR: No kaggle.json found")
    print("Recommended to place it in current directory: {current_dir_kaggle}")
    print("\nDownload it from: https://www.kaggle.com/settings")
    print("After placing the file, run this script again and it will automatically set secure permissions.")

Found kaggle.json in current directory
Location: /Users/admin/Projects/Programming/blood-cell-classification/kaggle.json
Set secure permissions (600) on kaggle.json


# Download dataset

Download the "Blood Cell Images" dataset from Kaggle and extract it to a directory named "dataset".

In [9]:
import os

os.makedirs("dataset", exist_ok=True)
print("Dataset directory created successfully!")

Dataset directory created successfully!


## Download the dataset

Use the Kaggle API to download the specified dataset.


In [23]:
import kaggle

# Download the dataset using Python API
kaggle.api.dataset_download_files('draaslan/blood-cell-detection-dataset', path='.', unzip=False)
print("Dataset downloaded successfully!")

Dataset URL: https://www.kaggle.com/datasets/draaslan/blood-cell-detection-dataset
Dataset downloaded successfully!


Unzip the folder

In [ ]:
import zipfile
import os

# Get current directory and construct the zip file path
zip_path = os.path.join(os.getcwd(), "blood-cell-detection-dataset.zip")

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(os.getcwd(), "dataset"))